In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.probability import FreqDist
from nltk.util import ngrams
import pandas as pd

from gensim import corpora
import gensim

import ast
import fs

In [42]:
INTERIM_FILES_DIR = fs.open_fs("MCD-NLP-HPT/data/interim")
INTERIM_COMMENTS_CLEANED_BY_VIDEO_DIR = INTERIM_FILES_DIR.getsyspath("Comments_cleaned_only_sw.csv")

In [43]:
df_video_corpus = pd.read_csv(INTERIM_COMMENTS_CLEANED_BY_VIDEO_DIR)

In [35]:
df_video_corpus.columns

Index(['id', 'videoId', 'textOriginal', 'authorDisplayName', 'likeCount',
       'publishedAt', 'category_id', 'category_description', 'Tokens',
       'Tokens_without_stopwords'],
      dtype='object')

In [36]:
# Convert to list
df_video_corpus['Tokens_without_stopwords_nose']  = df_video_corpus['Tokens_without_stopwords'].apply(ast.literal_eval)
df_video_corpus.head(5)

,id,videoId,textOriginal,authorDisplayName,likeCount,publishedAt,category_id,category_description,Tokens,Tokens_without_stopwords,Tokens_without_stopwords_nose
0,Ugz0QmXyVLSM0nUK7kB4AaABAg,cQ9RLDhq6JY,me gusta su episodio pero solo he escuchado lo...,@fabianbecerra2272,0,2024-07-15T12:41:43Z,1,Quejas o sugerencias de mejora,"['me', 'gusta', 'su', 'episodio', 'pero', 'sol...","['episodio', 'escuchado', 'youtube', 'spotify'...","[episodio, escuchado, youtube, spotify, youtube]"
1,UgyJhRBhVcbyStZdxjR4AaABAg,cQ9RLDhq6JY,weeeeey,@SantiagoItzcoatl,0,2024-05-26T21:41:04Z,7,Comentarios generales,['weeeeey'],[],[]
2,UgzT1q2am7kMEM5ckDp4AaABAg,cQ9RLDhq6JY,les voy a enviar una foto de tarapaca,@Sonyeke_1,0,2024-05-23T22:06:21Z,7,Comentarios generales,"['les', 'voy', 'a', 'enviar', 'una', 'foto', '...","['foto', 'tarapaca']","[foto, tarapaca]"
3,Ugyckucg38lqZFdZhaR4AaABAg,cQ9RLDhq6JY,entonces teca esta solterooooooooooooooo woooooo,@alejandranavarro464,0,2024-05-05T09:24:57Z,6,Comentarios humorísticos o memes,"['entonces', 'teca', 'esta', 'solteroooooooooo...",['teca'],[teca]
4,UgzhZHVt-tEbDpbCvtB4AaABAg,cQ9RLDhq6JY,me encantan sus episodio los escucho en spotif...,@fiorellamartinez873,0,2024-04-21T21:09:08Z,5,Felicitaciones y agradecimientos,"['me', 'encantan', 'sus', 'episodio', 'los', '...","['encantan', 'episodio', 'spotify', 'viajo', '...","[encantan, episodio, spotify, viajo, conoci, o..."


In [17]:
# df_video_corpus['List_of_Tokens_combined'][0]

In [37]:
from gensim import corpora
documents = df_video_corpus['Tokens_without_stopwords_nose'].tolist()
from gensim import corpora

# Crear el diccionario
dictionary = corpora.Dictionary(df_video_corpus['Tokens_without_stopwords_nose'])

# Ver el diccionario (opcional)
print(dictionary.token2id)  # Muestra la correspondencia palabra-ID

{'episodio': 0, 'escuchado': 1, 'spotify': 2, 'youtube': 3, 'foto': 4, 'tarapaca': 5, 'teca': 6, 'argentina': 7, 'conoci': 8, 'encantan': 9, 'onda': 10, 'saludos': 11, 'viajo': 12, 'colombia': 13, 'comentario': 14, 'dejo': 15, 'lol': 16, 'guerra': 17, 'imperio': 18, 'japones': 19, 'medio': 20, 'parte': 21, 'vs': 22, 'encanta': 23, 'historia': 24, 'vida': 25, 'napoleon': 26, 'cubana': 27, 'republica': 28, 'chamba': 29, 'favoritos': 30, 'camino': 31, 'compania': 32, 'ojala': 33, 'salga': 34, 'sido': 35, 'siguiente': 36, 'universidad': 37, 'adoro': 38, 'fan': 39, 'mega': 40, 'buena': 41, 'clase': 42, 'dije': 43, 'ingles': 44, 'maestro': 45, 'chile': 46, 'ciudades': 47, 'chicos': 48, 'contenido': 49, 'mma': 50, 'anos': 51, 'broma': 52, 'creer': 53, 'horas': 54, 'informacion': 55, 'mejor': 56, 'oyendo': 57, 'tres': 58, 'casa': 59, 'bolivia': 60, 'chingon': 61, 'divierto': 62, 'francia': 63, 'haciendo': 64, 'increibles': 65, 'mexico': 66, 'barba': 67, 'escuchaba': 68, 'igual': 69, 'imaginaba

In [38]:
# Crear el corpus
corpus = [dictionary.doc2bow(doc) for doc in documents]

# Ver el corpus (opcional)
print(corpus)  # Ejemplo: [(0, 1), (1, 1)] -> palabras con su frecuencia

[[(0, 1), (1, 1), (2, 1), (3, 2)], [], [(4, 1), (5, 1)], [(6, 1)], [(0, 1), (2, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)], [(0, 1), (13, 1), (14, 1), (15, 1), (16, 1)], [(0, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)], [], [], [(13, 1), (23, 1), (24, 2), (25, 1)], [(21, 1), (26, 1)], [(16, 1), (27, 1), (28, 1)], [(0, 1), (24, 1), (29, 1)], [(0, 2), (21, 1), (30, 1)], [(0, 1), (14, 1), (24, 1)], [(0, 1), (11, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1)], [(0, 2), (11, 1), (24, 1), (38, 1), (39, 2), (40, 1)], [(23, 2)], [(0, 1), (41, 1), (42, 2), (43, 1), (44, 2), (45, 1)], [(5, 1), (46, 1), (47, 1)], [(23, 1), (24, 1), (48, 1), (49, 1), (50, 1)], [(0, 4), (23, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1)], [(11, 1), (18, 1), (19, 1), (21, 1), (59, 1)], [(24, 1), (60, 1)], [(14, 1)], [(0, 1), (11, 1), (23, 2), (39, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1)], [], [(0, 1), (16, 2), (23, 1), (67, 1

# Model

In [60]:
from gensim.models import LdaModel

# Crear el modelo LDA
lda_model = LdaModel(corpus, num_topics=3, id2word=dictionary, passes=5)

In [62]:
# Ver los tópicos generados
topics = lda_model.print_topics(num_words=30)  # Muestra las 4 palabras más representativas de cada tópico
for topic in topics:
    print(topic)

(0, '0.034*"guerra" + 0.033*"saludos" + 0.019*"paraguay" + 0.014*"anos" + 0.013*"bien" + 0.012*"argentina" + 0.012*"pais" + 0.011*"brasil" + 0.010*"chile" + 0.010*"gobierno" + 0.008*"anime" + 0.008*"lopez" + 0.008*"ninos" + 0.008*"episodio" + 0.008*"mal" + 0.007*"dato" + 0.007*"gente" + 0.006*"mundial" + 0.006*"morales" + 0.006*"vida" + 0.006*"evo" + 0.006*"cesar" + 0.005*"america" + 0.005*"chaco" + 0.005*"colombia" + 0.005*"julio" + 0.005*"unas" + 0.005*"carrera" + 0.005*"campo" + 0.005*"grande"')
(1, '0.125*"episodio" + 0.049*"historia" + 0.021*"encanta" + 0.020*"saludos" + 0.019*"parte" + 0.015*"mejor" + 0.014*"spotify" + 0.014*"imperio" + 0.014*"youtube" + 0.013*"iker" + 0.012*"japones" + 0.012*"buen" + 0.011*"mexico" + 0.011*"tema" + 0.010*"comentario" + 0.009*"fan" + 0.009*"amo" + 0.008*"bien" + 0.007*"contenido" + 0.007*"interesante" + 0.006*"excelente" + 0.006*"teca" + 0.006*"israel" + 0.006*"anos" + 0.006*"escuchando" + 0.005*"teotihuacan" + 0.005*"minutos" + 0.005*"verdad" + 

# Review distribution

In [63]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

# Visualización interactiva
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)

# Guardando modelo, diccionario, corpus

In [66]:
lda_model.save("topic_model_ok_final.gensim")
#from gensim.models.ldamodel import LdaModel
#lda_model_cargado = LdaModel.load("topic_model_yt.gensim")

In [67]:
dictionary.save("topic_model_ok_final.dict")

In [68]:
from gensim.corpora import MmCorpus
MmCorpus.serialize("topic_model_ok_final.mm", corpus)

(0): Política, conflictos y contexto

Palabras clave: guerra, saludos, paraguay, argentina, brasil, chile, gobierno, mundial, evo, cesar, chaco, colombia.

Interpretación: Este tema aborda conflictos históricos y políticos, incluye menciones generales sobre la guerra, datos históricos y aspectos sociales.

---

(1): Admiración por los episodios y narrativa histórica

Palabras clave: episodio, historia, encanta, saludos, mejor, imperio, japonés, youtube, spotify, teotihuacan, tema, fan.

Interpretación: Este tema refleja la respuesta positiva de la audiencia hacia los episodios, mostrando admiración (encanta, fan, amo). También hay referencias a temas históricos específicos, como imperios y Teotihuacán, junto con menciones a plataformas donde se comparte contenido (Spotify, YouTube).

---

(2): Participación de creadores y temas culturales

Palabras clave: teca, iker, episodio, invitado, teotihuacan, jay, mayas, audio, casa, amo.

Interpretación: Este tema se centra en la participación de los creadores (Teca, Iker) y menciones de invitados en los episodios. Incluye referencias culturales como las pirámides de Teotihuacán y los mayas, además de aspectos relacionados con la producción del contenido (audio, tiempo, casa).

---

Resumen de los tres temas:

1. (0): Política y conflictos históricos (*en América Latina).

2. (1): Interacción con la audiencia y admiración por la narrativa histórica.

3. (2): Participación de creadores e interés en temas culturales y de producción.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2820e488-6f1b-466d-af14-a66826f012e3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>